In [ ]:
import re

lines = open('test.txt').read().split("\n")

a = int(re.search(r'(\d+)', lines[0]).group())
b = int(re.search(r'(\d+)', lines[1]).group())
c = int(re.search(r'(\d+)', lines[2]).group())
program = list(map(int, re.findall(r'(\d+)', lines[4])))
pointer = 0


print(a, b, c, program)


def get_combo(operand):
    global a 
    global b 
    global c
    if operand <= 3:
        return operand
    elif operand == 4:
        return a
    elif operand == 5:
        return b
    elif operand == 6:
        return c

output = []

while pointer < len(program):
    jumped = False

    opcode = program[pointer]
    literal = program[pointer + 1]

    combo = get_combo(literal)

    if opcode == 0:
        # adv, divide a by 2 to the power of combo, save in a
        a = a // 2 ** combo
    elif opcode == 1:
        # bxl, xor b with literal, save in b
        b = b ^ literal
    elif opcode == 2:
        # bst, combo modulo 8, save in b
        b = combo % 8
    elif opcode == 3:
        # jnz, jump by literal if a is not 0
        if a != 0:
            pointer = literal
            jumped = True
    elif opcode == 4:
        # bxc, xor b and c, save in b
        b = b ^ c
    elif opcode == 5:
        # out, output combo modulo 8
        output.append(combo % 8)
    elif opcode == 6:
        #bdv, divide a by 2 ** combo, save in b
        b = a // 2 ** combo
    elif opcode == 7:
        # cdv, divide a by 2 ** combo, save in c
        c = a // 2 ** combo
    

    pointer += 2 if not jumped else 0
    

print(a, b, c, program)
print(output)

print(output == program)

print( ",".join(map(str, output)) )

In [ ]:
import re
import random
import matplotlib.pyplot as plt
import functools

def plot_scores(scores):
    x = list(scores.keys())
    y = list(scores.values())

    plt.figure()
    plt.plot(x,y)
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title('Plot of Scores')
    plt.grid(True)
    plt.show()

def ends_with(list, sublist):
    if len(sublist) > len(list):
        return False
    return list[-len(sublist):] == sublist

def starts_with(list, sublist):
    if len(sublist) > len(list):
        return False
    return list[:len(sublist)] == sublist

def get_combo(operand, a, b, c):
    if operand <= 3:
        return operand
    elif operand == 4:
        return a
    elif operand == 5:
        return b
    elif operand == 6:
        return c

# @functools.lru_cache(maxsize=None)
def run_program(program, a):
    b = 0
    c = 0
    pointer = 0
    output = tuple()

    while pointer < len(program):
        jumped = False

        opcode = program[pointer]
        literal = program[pointer + 1]

        combo = get_combo(literal, a, b, c)

        if opcode == 0:
            # adv, divide a by 2 to the power of combo, save in a
            a = a // 2 ** combo
        elif opcode == 1:
            # bxl, xor b with literal, save in b
            b = b ^ literal
        elif opcode == 2:
            # bst, combo modulo 8, save in b
            b = combo % 8
        elif opcode == 3:
            # jnz, jump by literal if a is not 0
            if a != 0:
                pointer = literal
                jumped = True
        elif opcode == 4:
            # bxc, xor b and c, save in b
            b = b ^ c
        elif opcode == 5:
            # out, output combo modulo 8
            output += (combo % 8, )
        elif opcode == 6:
            #bdv, divide a by 2 ** combo, save in b
            b = a // 2 ** combo
        elif opcode == 7:
            # cdv, divide a by 2 ** combo, save in c
            c = a // 2 ** combo
        
        pointer += 2 if not jumped else 0

    return output

def to_int(program):
    return int("".join(map(str, program[::-1])))

def check(program, out):
    # if len(out) != len(program):
    #     return None

    p = to_int(program)
    o = to_int(out)

    return p - o

def search(low, high, steps):
    global scores

    step = (high - low) // steps

    if step == 0:
        step = 1

    for i in range(low, high + step, step):
        out_1 = run_program(program, i)
        score_1 = check(program, out_1)

        out_2 = run_program(program, i + step)
        score_2 = check(program, out_2)

        scores[i] = score_1

        # print(f"Checking {i} - {i+step}")
        # print(f"Score 1: {score_1}")
        # print(f"Score 2: {score_2}")

        if len(out_1) != len(program) and len(out_2) != len(program):
            continue

        if score_1 == 0:
            return i

        if step > 1 and (
            (len(out_1) != len(out_2) and (len(out_1) == len(program) or len(out_2) == len(program))) or
            (score_1 < 0 and score_2 > 0 or score_1 > 0 and score_2 < 0)
            ):
            #search in between
            solution = search(i-step, i+2*step, steps)

            if solution is not None:
                return solution

    return None


lines = open('input.txt').read().split("\n")

a = int(re.search(r'(\d+)', lines[0]).group())
b = int(re.search(r'(\d+)', lines[1]).group())
c = int(re.search(r'(\d+)', lines[2]).group())
program = tuple(map(int, re.findall(r'(\d+)', lines[4])))
pointer = 0

i = 1
while True:
    out = run_program(program, i)
    if len(out) >= len(program):
        break
    i = i * 2

j = i
while True:
    out = run_program(program, j)
    if len(out) > len(program):
        break

    j = j * 2

scores = {}
low = i//4
high = j + i
print(low, high, high - low)

steps = 1000
ans = None
while ans is None:
    ans = search(low, high, steps)
    steps = steps * 2
    if ans is None:
        print(f"Not found, increasing steps to {steps}")
        print(f"Searched at {len(scores)} Positions.")
        print(run_program.cache_info())

# plot_scores(scores)
# print(scores)
print(ans)
